## Docker images

In [1]:
run(pipeline(`docker images`, `grep openresty`, `grep -v 127.0.1.1`));

openresty                            bionic-with-top     a82cdeb5f7e6        2 days ago          648MB
openresty/openresty                  bionic              de816c768659        2 weeks ago         562MB


In [2]:
;cat Dockerfile

# openresty image with ngxtop utility
#
# build using:
# docker build -t openresty:bionic-with-top -f Dockerfile .

FROM openresty/openresty:bionic

RUN apt-get install -y python-pip \
    && pip install ngxtop \
    && rm /usr/local/openresty/nginx/logs/access.log \
    && rm /usr/local/openresty/nginx/logs/error.log

COPY nginx.conf /usr/local/openresty/nginx/conf/nginx.conf


## Run a Nginx Webserver

Nginx is a very popular webserver. Applications like webservers are interesting in the sense that they are services. That is, they expose network endpoints that can be called from other applications. We shall see how to expose and use network ports or service endpoints from applications in Kubernetes.

In [3]:
using Kuber
ctx = KuberContext();
Kuber.set_api_versions!(ctx);

┌ Info: unsupported crd.projectcalico.org/v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:131


We need to give a specification of the application, the entity of type `Pod`, to the cluster. The `Pod` specifies which image it would execute, and adds some metadata to the entity.

Because this is a service, we also specify the network port. And we also create another entity called a `Service` in the cluster that exposes this port as a callable endpoint.

The `Kuber.kuber_obj` method converts a JSON specification to an entity object.

In [4]:
nginx_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "nginx-pod",
               "namespace": "default",
               "labels": {
                   "name": "nginx-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "nginx",
                   "image": "openresty:bionic-with-top",
                   "ports": [{"containerPort": 80}]
               }]
           }
       }""");

In [5]:
nginx_service = kuber_obj(ctx, """{
           "kind": "Service",
           "metadata": {
               "name": "nginx-service",
               "namespace": "default",
               "labels": {"name": "nginx-service"}
           },
           "spec": {
               "type": "NodePort",
               "ports": [{"port": 80}],
               "selector": {"name": "nginx-pod"}
           }
       }""");

In [6]:
typeof(nginx_pod), typeof(nginx_service)

(Kuber.Kubernetes.IoK8sApiCoreV1Pod, Kuber.Kubernetes.IoK8sApiCoreV1Service)

We can create the entities in the cluster by applying the `put!` verb on them.

In [7]:
pod = put!(ctx, nginx_pod);

In [8]:
service = put!(ctx, nginx_service);

We can now see our pod in our namespace.
And we can fetch them back from the cluster using the `get` verb on the entity name.

In [9]:
collect(item.metadata.name for item in (get(ctx, :Pod)).items)

1-element Array{String,1}:
 "nginx-pod"

In [10]:
collect(item.metadata.name for item in (get(ctx, :Service)).items)

2-element Array{String,1}:
 "kubernetes"
 "nginx-service"

In [11]:
get(ctx, :Service, "nginx-service")

{
  "kind": "Service",
  "metadata": {
    "creationTimestamp": "2020-05-08T03:17:22Z",
    "uid": "f3d63ad4-fa6c-4cc1-a47a-e6cd311a8b17",
    "labels": {
      "name": "nginx-service"
    },
    "selfLink": "/api/v1/namespaces/default/services/nginx-service",
    "name": "nginx-service",
    "managedFields": [
      {
        "fieldsType": "FieldsV1",
        "apiVersion": "v1",
        "manager": "HTTP.jl",
        "fieldsV1": {},
        "operation": "Update",
        "time": "2020-05-08T03:17:22Z"
      }
    ],
    "namespace": "default",
    "resourceVersion": "6434"
  },
  "apiVersion": "v1",
  "status": {
    "loadBalancer": {}
  },
  "spec": {
    "externalTrafficPolicy": "Cluster",
    "clusterIP": "10.103.36.245",
    "sessionAffinity": "None",
    "selector": {
      "name": "nginx-pod"
    },
    "ports": [
      {
        "nodePort": 30605,
        "port": 80,
        "protocol": "TCP",
        "targetPort": "80"
      }
    ],
    "type": "NodePort"
  }
}


In the service entity that we fetched, we can now see the IP and port of the nginx service. The IP is a virtual IP and was assigned automatically by Kubernetes.

We can use our webserver now.

In [12]:
;curl http://10.103.36.245/

<!DOCTYPE html>
<html>
<head>
<title>Welcome to OpenResty!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to OpenResty!</h1>
<p>If you see this page, the OpenResty web platform is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="https://openresty.org/">openresty.org</a>.<br/>
Commercial support is available at
<a href="https://openresty.com/">openresty.com</a>.</p>

<p><em>Thank you for flying OpenResty.</em></p>
</body>
</html>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   649  100   649    0     0   633k      0 --:--:-- --:--:-- --:--:--  633k


At this point we can use the Kubernetes dashboard to get inside the container. Kubernetes dashboard is a useful interactive dashboard, usually available by default in all Kubernetes clusters.

Finally when we are done, we can use the `delete!` verb to remove the entities from the cluster. This will stop the process that the cluster was running.

In [13]:
delete!(ctx, service);
delete!(ctx, pod);